# Ejercicio 03: Cálculo de Relevancia y Ranking de Documentos
El objetivo de este ejercicio es calcular analíticamente la relevancia de cada documento en un corpus y luego ordenar (rankear) los documentos basándonos en esa puntuación de relevancia para tres consultas específicas. Seguirás los siguientes pasos:

Descripción del Ejercicio

1. Procesamiento del Corpus:

* Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor y fecha).
2. Procesamiento de las Consultas:

* Definir las consultas proporcionadas.
* Extraer las palabras clave de cada consulta.
3. Cálculo de Relevancia:

* Utilizar métricas de similitud (Similitud Coseno y Jaccard) entre la representación vectorial de los documentos y las de las consultas.
* Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.
4. Ranking de Documentos:

* Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.
* Mostrar el ranking de documentos para cada consulta.

 ### 1.- Procesamiento del Corpus

- Leer y parsear el archivo XML proporcionado que contiene el corpus de documentos con sus metadatos (keywords, autor ya. fecha).
fecha).

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

# Función para leer y parsear el archivo XML
def procesar_corpus_xml(archivo_xml):
    # Parsear el archivo XML
    tree = ET.parse(archivo_xml)
    root = tree.getroot()

    # Lista para almacenar los documentos
    documentos = []

    # Iterar sobre cada elemento "document" en el XML
    for doc in root.findall('document'):
        # Extraer metadatos del documento
        texto = doc.find('text').text.strip() if doc.find('text') is not None else ''
        autor = doc.find('author').text.strip() if doc.find('author') is not None else 'Desconocido'
        fecha = doc.find('date').text.strip() if doc.find('date') is not None else 'No especificada'
        keywords = doc.find('keywords').text.strip() if doc.find('keywords') is not None else 'Sin keywords'

        # Agregar el documento a la lista
        documentos.append({
            'texto': texto,
            'autor': autor,
            'fecha': fecha,
            'keywords': keywords
        })

    # Convertir la lista de documentos en un DataFrame de Pandas
    df = pd.DataFrame(documentos)
    return df

# Uso de la función
archivo_xml = '/content/03ranking_corpus.xml'
df_corpus = procesar_corpus_xml(archivo_xml)

# Mostrar una muestra del DataFrame procesado
print(df_corpus.head())

  texto             autor       fecha  \
0          Dr. Juan Pérez  2023-01-15   
1        Dra. María López  2023-02-10   
2        Miguel Rodríguez  2023-03-05   
3          Lucía Martínez  2023-04-20   
4        Carlos Fernández  2023-05-15   

                                            keywords  
0  telemedicina, salud crónica, tratamiento, tecn...  
1  nutrición, rendimiento académico, salud mental...  
2  amistad, bienestar estudiantil, campus, relaci...  
3  bibliotecas universitarias, investigación, aca...  
4  espacios verdes, campus universitario, concent...  


### 2.- Procesamiento de las Consultas:

- Definir las consultas proporcionadas.

In [2]:
# Definir las consultas
queries = {
    1: "Impacto de la salud mental en el rendimiento académico de los estudiantes universitarios",
    2: "Actividades extracurriculares y bienestar emocional en el campus universitario",
    3: "Estrategias universitarias para reducir el estrés en estudiantes"
}

- Extraer las palabras clave de cada consulta.

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Cargar stopwords en español
stop_words = set(stopwords.words('spanish'))

# Definir función para extraer palabras clave
def extraer_palabras_clave(consulta, stop_words):
    # Convertir a minúsculas
    consulta = consulta.lower()
    # Tokenizar la consulta (solo palabras alfanuméricas)
    tokens = word_tokenize(consulta)
    # Filtrar tokens: eliminar stopwords y caracteres especiales
    palabras_clave = [word for word in tokens if word.isalpha() and word not in stop_words]
    return palabras_clave

# Crear diccionario para almacenar las palabras clave de cada consulta
palabras_clave_queries = {}

# Extraer palabras clave de cada consulta
for query_id, query_text in queries.items():
    palabras_clave = extraer_palabras_clave(query_text, stop_words)
    palabras_clave_queries[query_id] = palabras_clave

# Mostrar las palabras clave extraídas de cada consulta
print("Palabras clave extraídas por consulta:")
for query_id, keywords in palabras_clave_queries.items():
    print(f"Consulta {query_id}: {keywords}")

Palabras clave extraídas por consulta:
Consulta 1: ['impacto', 'salud', 'mental', 'rendimiento', 'académico', 'estudiantes', 'universitarios']
Consulta 2: ['actividades', 'extracurriculares', 'bienestar', 'emocional', 'campus', 'universitario']
Consulta 3: ['estrategias', 'universitarias', 'reducir', 'estrés', 'estudiantes']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### 3.- Cálculo de Relevancia:

- Utilizar métricas de similitud Coseno entre la representación vectorial de los documentos y las de las consultas.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import binarize
import pandas as pd
import numpy as np

# Crear el vocabulario usando el contenido completo de documentos y consultas
# Extraer textos y concatenar con keywords para obtener mejor vocabulario
docs_content = (df_corpus['texto'] + " " + df_corpus['keywords']).tolist()
queries_content = [" ".join(palabras_clave_queries[q]) for q in palabras_clave_queries]

# Lista combinada de textos de documentos y textos de consultas
all_texts = docs_content + queries_content

# Vectorizador TF-IDF para todo el vocabulario generado
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(all_texts)

# Separar matriz TF-IDF en documentos y consultas
tfidf_docs = tfidf_matrix[:len(docs_content)]
tfidf_queries = tfidf_matrix[len(docs_content):]

In [5]:
# Calcular Similitud del Coseno
cosine_sim_matrix = cosine_similarity(tfidf_queries, tfidf_docs)

- Utilizar métricas de similitud Jaccard entre la representación vectorial de los documentos y las de las consultas.

In [6]:
# Calcular Similitud de Jaccard
def jaccard_similarity(vector1, vector2):
    # Convertir a binario
    vector1 = binarize(vector1.toarray())[0]
    vector2 = binarize(vector2.toarray())[0]
    # Calcular similitud de Jaccard
    intersection = np.sum(np.minimum(vector1, vector2))
    union = np.sum(np.maximum(vector1, vector2))
    return intersection / union if union != 0 else 0

# Crear matriz de similitud de Jaccard
jaccard_sim_matrix = np.zeros((len(queries_content), len(docs_content)))

for i, query_vector in enumerate(tfidf_queries):
    for j, doc_vector in enumerate(tfidf_docs):
        jaccard_sim_matrix[i, j] = jaccard_similarity(query_vector, doc_vector)


- Calcular la puntuación de relevancia para cada documento del corpus respecto a cada consulta.

In [7]:
# Crear DataFrames para ambas métricas
cosine_df = pd.DataFrame(
    cosine_sim_matrix,
    index=[f"Consulta {i+1}" for i in range(len(queries_content))],
    columns=[f"Documento {j+1}" for j in range(len(docs_content))]
)

jaccard_df = pd.DataFrame(
    jaccard_sim_matrix,
    index=[f"Consulta {i+1}" for i in range(len(queries_content))],
    columns=[f"Documento {j+1}" for j in range(len(docs_content))]
)

# --- Mostrar Resultados ---
print("Matriz de Similitud del Coseno:")
print(cosine_df)

print("\nMatriz de Similitud de Jaccard:")
print(jaccard_df)

Matriz de Similitud del Coseno:
            Documento 1  Documento 2  Documento 3  Documento 4  Documento 5  \
Consulta 1     0.095716     0.644617     0.000000     0.000000     0.000000   
Consulta 2     0.000000     0.000000     0.227196     0.000000     0.303344   
Consulta 3     0.000000     0.054051     0.000000     0.199459     0.147081   

            Documento 6  Documento 7  Documento 8  Documento 9  Documento 10  \
Consulta 1     0.048043     0.725969     0.041472          0.0           0.0   
Consulta 2     0.096251     0.000000     0.213397          0.0           0.0   
Consulta 3     0.046476     0.000000     0.040119          0.0           0.0   

            ...  Documento 21  Documento 22  Documento 23  Documento 24  \
Consulta 1  ...      0.270267           0.0      0.147657      0.047237   
Consulta 2  ...      0.000000           0.0      0.316191      0.000000   
Consulta 3  ...      0.000000           0.0      0.045091      0.260243   

            Documento 25  Doc

### 4.- Ranking de Documentos:

- Ordenar los documentos en función de su puntuación de relevancia de mayor a menor.

In [8]:
# Generar el ranking de relevancia
def obtener_ranking(similarity_df):
    rankings = {}
    for query_idx in similarity_df.index:
        # Ordenar documentos por relevancia de mayor a menor
        ranking = similarity_df.loc[query_idx].sort_values(ascending=False)
        rankings[query_idx] = ranking
    return rankings

# Obtener rankings de Similitud del Coseno
cosine_rankings = obtener_ranking(cosine_df)

# Obtener rankings de Similitud de Jaccard
jaccard_rankings = obtener_ranking(jaccard_df)

- Mostrar el ranking de documentos para cada consulta.

In [9]:
# Función para imprimir el ranking de documentos
def mostrar_ranking(rankings, metodo):
    print(f"\nRanking de documentos por {metodo}:\n")
    for query, ranking in rankings.items():
        print(f"{query}:")
        for doc, score in ranking.items():
            print(f"  {doc} -> Relevancia: {score:.4f}")

# Mostrar ranking por Similitud del Coseno
mostrar_ranking(cosine_rankings, "Similitud del Coseno")

# Mostrar ranking por Similitud de Jaccard
mostrar_ranking(jaccard_rankings, "Similitud de Jaccard")


Ranking de documentos por Similitud del Coseno:

Consulta 1:
  Documento 7 -> Relevancia: 0.7260
  Documento 2 -> Relevancia: 0.6446
  Documento 14 -> Relevancia: 0.5415
  Documento 11 -> Relevancia: 0.4370
  Documento 13 -> Relevancia: 0.3493
  Documento 18 -> Relevancia: 0.2908
  Documento 21 -> Relevancia: 0.2703
  Documento 19 -> Relevancia: 0.2320
  Documento 29 -> Relevancia: 0.1653
  Documento 12 -> Relevancia: 0.1620
  Documento 23 -> Relevancia: 0.1477
  Documento 1 -> Relevancia: 0.0957
  Documento 15 -> Relevancia: 0.0541
  Documento 6 -> Relevancia: 0.0480
  Documento 24 -> Relevancia: 0.0472
  Documento 27 -> Relevancia: 0.0441
  Documento 20 -> Relevancia: 0.0429
  Documento 8 -> Relevancia: 0.0415
  Documento 26 -> Relevancia: 0.0379
  Documento 25 -> Relevancia: 0.0000
  Documento 28 -> Relevancia: 0.0000
  Documento 16 -> Relevancia: 0.0000
  Documento 22 -> Relevancia: 0.0000
  Documento 17 -> Relevancia: 0.0000
  Documento 10 -> Relevancia: 0.0000
  Documento 9 -> R